In [ ]:
#!pip install pandarallel
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Prediction set

In [ ]:
# test_data = pd.read_csv("/content/drive/MyDrive/Amazon ML Hackathon/dataset/test.csv", escapechar = "\\", quoting = 3)
# pd.DataFrame(test_data)

# Code

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Amazon ML Hackathon/dataset/train.csv", escapechar = "\\", quoting = 3)
data = pd.DataFrame(data)

In [ ]:
data

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,NaN,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),NaN,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,NaN,NaN,NaN,4
...,...,...,...,...,...
2903019,Premium Aviator Sunglasses - HD Polarized (Bri...,These premium Aviator Sunglasses with 5 color ...,"[Frame size: Lens height - 56mm, Lens width - ...",Generic,1040
2903020,Social Distance Stickers - Set of 5 Sticker Sl...,set of 5 prints social distancing sticker self...,[covid19 safety sticker - set of 5 to maintain...,Generic,15199
2903021,Torr-to Face Shield PACK OF 5 with Adjustable ...,* COMPLETE FACE PROTECTION: Torr-to Face Shiel...,"[350 MICRONS PACK OF 5 PCS,COMPLETE FACE PROTE...",TORR-TO,1044933
2903022,Type-C to 3.5 MM for Oppo R17 Pro Type-C to 3....,Still want to use your favorite earphones/head...,"[Indian Connectors: Made for Indian sockets, t...",SHOPBELL,14790


## Data Cleaning

In [ ]:
#Dropping the description column
data = data.drop("DESCRIPTION", axis = 1)

#Removing duplicate titles but ensuring we dont drop any unique browse node id
data = data.drop_duplicates(subset=["TITLE","BROWSE_NODE_ID"])

#Dropping the null titles
data = data.dropna(subset=["TITLE"])

data.BULLET_POINTS = data.BULLET_POINTS.fillna(" ")
# data["TBP"] = data["TITLE"].astype(str) + " " +data["BULLET_POINTS"].astype(str)

# data = data.drop(columns=["BULLET_POINTS"])
data.BRAND = data.BRAND.fillna(" ")
data["TBP"] = data["BRAND"].astype(str) + " " +data["BULLET_POINTS"].astype(str)

data = data.drop(columns=["BULLET_POINTS", "BRAND"])

In [ ]:
data, data.shape

(                                                     TITLE  ...                                                TBP
 0               Pete The Cat Bedtime Blues Doll, 14.5 Inch  ...  MerryMakers [Pete the Cat Bedtime Blues plush ...
 1        The New Yorker NYHM014 Refrigerator Magnet, 2 ...  ...  The New Yorker [Cat In A Tea Cup by New Yorker...
 2        The Ultimate Self-Sufficiency Handbook: A Comp...  ...                         imusti Skyhorse Publishing
 3         Amway Nutrilite Kids Chewable Iron Tablets (100)  ...  Amway [Nutrilite Kids,Chewable Iron Tablets,Qu...
 4        Teacher Planner Company A4 6 Lesson Academic T...  ...                                                   
 ...                                                    ...  ...                                                ...
 2903019  Premium Aviator Sunglasses - HD Polarized (Bri...  ...  Generic [Frame size: Lens height - 56mm, Lens ...
 2903020  Social Distance Stickers - Set of 5 Sticker Sl...  ...  Generi

In [ ]:
def string_cleaning(string):
    string = list(string.lower())
    for i in string:
      if i == ",":
        string[string.index(i)] = " "
      if i in "[]:;":
        string.remove(i)
    string = "".join(string)
    output = []
    string = string.split(" ")
    for i in string:
      if i not in output:
        output.append(i)
    output = " ".join(output)
    
    return output

data.TBP = data.TBP.apply(string_cleaning)

In [ ]:
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [ ]:
def remove_stopwords(string):
    stop_words = set(stopwords.words('english'))  
    word_tokens = word_tokenize(string)    
        
    filtered_sentence = [] 
      
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 

    return " ".join(filtered_sentence)

data.TBP = data.TBP.apply(remove_stopwords)


In [ ]:
data.to_csv("/content/drive/MyDrive/Amazon ML Hackathon/checkpoint_nltk.csv")

## word2vec


In [ ]:
# import gensim.downloader as api
# wv=api.load('word2vec-google-news-300')

In [ ]:
# TBP=data.TBP.tolist()
# #TITLE=data.TITLE.tolist()

In [ ]:
# from gensim.models.callbacks import CallbackAny2Vec
# from gensim.models import Word2Vec

# # init callback class
# class callback(CallbackAny2Vec):
#     """
#     Callback to print loss after each epoch
#     """
#     def __init__(self):
#         self.epoch = 0

#     def on_epoch_end(self, model):
#         loss = model.get_latest_training_loss()
        
#         if self.epoch == 0:
#             print('Loss after epoch {}: {}'.format(self.epoch, loss))
#         elif self.epoch % 1 == 0:
#             print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        
        
#         self.epoch += 1
#         self.loss_previous_step = loss

In [ ]:
# def train_data(sentences):
#     w2v_model = Word2Vec(size = 300,
#                         window = 20,
#                         min_count = 2,
#                         workers = 20,
#                         sg = 0,
#                         negative = 5,
#                         sample = 1e-5)

#     w2v_model.build_vocab(sentences)

      
#     # train the w2v model
#     w2v_model.train(sentences, 
#                     total_examples=w2v_model.corpus_count, 
#                     epochs=100, 
#                     report_delay=1,
#                     compute_loss = True, # set compute_loss = True
#                     callbacks=[callback()]) # add the callback class
                  
#     return w2v_model

In [ ]:
# model_TBP=train_data(TBP)
# #model_title=train_data(TITLE)
# model_TBP.save("/content/drive/MyDrive/Amazon ML Hackathon/w2vtbp.model")
# #model_title.save("/content/drive/MyDrive/Amazon ML Hackathon/w2vtitle.model")

In [ ]:
# #encoded_docs = [[model_TBP.wv[word] for word in post] for post in TBP]
# model_TBP = Word2Vec.load("/content/drive/MyDrive/Amazon ML Hackathon/w2vtbp.model")


In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv("/content/drive/MyDrive/Amazon ML Hackathon/checkpoint_nltk.csv")
data.TBP = data.TBP.fillna("0")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## hashin vectorizer


In [ ]:
hash_length = 100

from sklearn.feature_extraction.text import HashingVectorizer

titles = data.TITLE.tolist()
tbp = data.TBP.tolist()

vectorizer = HashingVectorizer(norm = None,n_features = hash_length)
vectorizer.fit(titles, tbp)
titles = vectorizer.transform(titles)
tbp = vectorizer.transform(tbp)

In [ ]:
tbp = np.array(tbp, dtype = np.float16)

ValueError: ignored

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import *

nClasses = 9919
input = Input(shape =(300,2))

#YOLO CNN Architecture
conv = Conv1D(64,7, strides = 2, activation = 'relu')(input)
maxp = MaxPool1D(strides = 2)(conv)

conv = Conv1D(192, 3, strides = 2, activation= 'relu')(maxp)
maxp = MaxPool1D(strides = 2)(conv)

conv = Conv1D(128,1, activation= 'relu')(maxp)
conv = Conv1D(256, 3, activation= 'relu')(conv)
conv = Conv1D(256, 1, activation= 'relu')(conv)
conv = Conv1D(512, 3, activation= 'relu')(conv)
maxp = MaxPool1D(strides = 2)(conv)

conv = Conv1D(256,1 ,activation= 'relu')(maxp)
conv = Conv1D(512, 3, activation = 'relu')(conv)
conv = Conv1D(256,1 ,activation= 'relu')(conv)
conv = Conv1D(512, 3, activation = 'relu')(conv)
conv = Conv1D(256,1 ,activation= 'relu')(conv)
conv = Conv1D(512, 3, activation = 'relu')(conv)
conv = Conv1D(256,1 ,activation= 'relu')(conv)
conv = Conv1D(512, 3, activation = 'relu')(conv)
conv = Conv1D(512, 1, activation = 'relu')(conv)
conv = Conv1D(1024, 3, activation = 'relu')(conv)
maxp = MaxPool1D(strides = 2)(conv)

conv = Conv1D(512,1 ,activation= 'relu')(maxp)
conv = Conv1D(1024, 3, activation = 'relu')(conv)
conv = Conv1D(512,1 ,activation= 'relu')(conv)
conv = Conv1D(1024, 3, activation = 'relu')(conv)
conv = Conv1D(1024, 3, activation = 'relu')(conv)
conv = Conv1D(1024, 3, strides = 2, activation = 'relu')(conv)

conv = Conv1D(1024, 3, activation = 'relu')(conv)
conv = Conv1D(1024, 3, activation = 'relu')(conv)

output = Dense(nClasses, activation = 'softmax')(conv)

model = keras.Model(inputs = input, outputs = output)
model.summary()

opt= keras.optimizers.Adam(learning_rate=1e-3)
model.compile(optimizer = opt, loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])


